In [5]:
import math

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import xgboost as xgb
from lightgbm import plot_importance
import joblib
from hnmchallenge.constant import *
from hnmchallenge.data_reader import DataReader
from hnmchallenge.evaluation.python_evaluation import map_at_k, recall_at_k
from hnmchallenge.feature_manager import FeatureManager
from hnmchallenge.datasets.last_week_last_week import LWLWDataset
from hnmchallenge.datasets.last_month_last_week_dataset import LMLWDataset
from hnmchallenge.datasets.last_month_last_day import LMLDDataset
from hnmchallenge.datasets.last_month_last_week_user import LMLUWDataset
from hnmchallenge.models.itemknn.itemknn import ItemKNN
import lightgbm as lgb
from hnmchallenge.datasets.last2month_last_day import L2MLDDataset
from hnmchallenge.datasets.all_items_last_month_last_day import AILMLDDataset
from hnmchallenge.datasets.all_items_last_month_last_week import AILMLWDataset


In [3]:
dataset = AILMLDDataset()
dr = DataReader()

In [4]:
base_load_path = dataset._DATASET_PATH / "lgbm_models"

In [1]:
VERSION = 0
#NAME = f"cutf_100_ItemKNN_tw_True_rs_False"
NAME = "dataset_iip"
#NAME = f"cutf_200_EASE_tw_True_rs_False_l2_0.1"
#NAME = f"cutf_100_EASE_tw_True_rs_True_l2_0.1"
#NAME = "cutf_100_TimePop_alpha_1.0"
DATASET = f"{NAME}_{VERSION}.feather"
MODEL_NAME = f"lgbm_{DATASET}.pkl"

In [2]:
model=joblib.load(base_load_path/MODEL_NAME)

NameError: name 'joblib' is not defined

In [3]:
fig, ax = plt.subplots(figsize=(20, 20))
plot_importance(model, ax=ax)
plt.show()

NameError: name 'plt' is not defined

In [179]:
model.best_iteration_

153

In [8]:
import re
features = pd.read_feather(dataset._DATASET_PATH / f"dataset_dfs/full/{DATASET}")
features = features.rename(columns=lambda x: re.sub("[^A-Za-z0-9_]+", "", x))

# Remember to use the bert_ntree_limit when taking the predictions

In [9]:
features.head()

,customer_id,article_id,TimePop_alpha_10_score,TimePop_alpha_10_rank,colour_group_code,i_sales_channel_score_2_x,i_sales_channel_score_1_x,department_no,garment_group_no,index_group_no,...,average,time_score,tdiff,times_item_bought,ItemKNN_score,ItemKNN_rank,graphical_appearance_no_gbm,index_code_gbm,index_group_name_gbm,product_group_name_gbm
0,0,2067,5.338943,1.0,9,0.846024,0.153976,1747,1009,2,...,2.5,1.0,0.004855,1.0,0.001713,1.0,0,7,4,3
1,0,2257,5.338941,2.0,73,0.935973,0.064027,1747,1009,2,...,2.5,1.0,0.004855,1.0,0.001367,3.0,0,7,4,3
2,0,1871,5.338941,3.0,71,0.835887,0.164113,1747,1009,2,...,2.5,1.0,0.004855,1.0,0.001608,2.0,0,7,4,3
3,0,6262,5.338940,4.0,9,0.991320,0.008680,1722,1009,1,...,2.5,1.0,0.004855,1.0,0.001009,4.0,0,0,0,3
4,0,11440,5.338939,5.0,11,0.986473,0.013527,1636,1005,1,...,2.5,1.0,0.004855,1.0,0.000765,9.0,0,0,0,0


In [10]:
features.columns

Index(['customer_id', 'article_id', 'TimePop_alpha_10_score',
       'TimePop_alpha_10_rank', 'colour_group_code',
       'i_sales_channel_score_2_x', 'i_sales_channel_score_1_x',
       'department_no', 'garment_group_no', 'index_group_no', 'popularity',
       'popularity_last_month', 'number_bought', 'perceived_colour_master_id',
       'perceived_colour_value_id', 'product_type_no', 'section_no', 'price',
       'avg_price_x', 'max_price', 'min_price', 'std_price', 'sales_factor',
       'i_sales_channel_score_2_y', 'i_sales_channel_score_1_y',
       'popularity_sales_channel_1', 'popularity_sales_channel_2', 'pop_25',
       'pop_25_40', 'pop_40_60', 'pop_60', 'price_mean', 'price_min',
       'price_max', 'active', 'age', 'club_member_status_num', 'Monthly',
       'NONE', 'None', 'Regularly', 'FN', 'avg_price_y', 'user_min_price',
       'user_max_price', 'user_tendency', 'user_tendency_lm',
       'user_average_buy_day', 'u_sales_channel_score_2',
       'u_sales_channel_score

In [11]:
cat = [
    "index_code_gbm",
    "product_group_name_gbm",
    "index_group_name_gbm",
    "graphical_appearance_no_gbm",
]

cat_index = [i for i, c in enumerate(features.columns) if c in cat]
print(cat_index)

for col in cat:
    features[col] = pd.Categorical(features[col])

[56, 57, 58, 59]


In [12]:
customer_article_df = features[[DEFAULT_USER_COL, DEFAULT_ITEM_COL]].copy()
X = features.drop([DEFAULT_USER_COL, DEFAULT_ITEM_COL], axis=1)

In [13]:
y_pred = model.predict(X, num_iteration=model.best_iteration_, n_jobs=72)

In [14]:
customer_article_df["predicted_score"] = y_pred

In [15]:
customer_article_df

,customer_id,article_id,predicted_score
0,0,2067,2.151094
1,0,2257,1.595641
2,0,1871,1.954697
3,0,6262,1.275684
4,0,11440,-1.203858
...,...,...,...
254063647,1371979,13794,-0.784724
254063648,1371979,2659,-1.598550
254063649,1371979,289,-1.548159
254063650,1371979,4065,-0.330556


In [16]:
sorted_scores = customer_article_df.sort_values([DEFAULT_USER_COL, "predicted_score"], ascending=[True, False])

In [17]:
sorted_scores.head(20)

,customer_id,article_id,predicted_score
0,0,2067,2.151094
2,0,1871,1.954697
1,0,2257,1.595641
3,0,6262,1.275684
121,0,1880,1.232110
111,0,5486,1.206525
81,0,141,0.333619
113,0,363,0.206466
10,0,761,0.130958
19,0,197,0.077851


In [18]:
sorted_scores_index = sorted_scores.reset_index(drop=True)

In [19]:
cutoff = sorted_scores_index.groupby(DEFAULT_USER_COL).size().values
i=0 
filter_indices = []
for cut in cutoff:
    filter_indices.extend(range(i, i+12))
    i=i+cut

In [20]:
cutoff.mean()

185.18028834239567

In [21]:
filter_indices

[0,
 1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 201,
 202,
 203,
 204,
 205,
 206,
 207,
 208,
 209,
 210,
 211,
 212,
 402,
 403,
 404,
 405,
 406,
 407,
 408,
 409,
 410,
 411,
 412,
 413,
 602,
 603,
 604,
 605,
 606,
 607,
 608,
 609,
 610,
 611,
 612,
 613,
 802,
 803,
 804,
 805,
 806,
 807,
 808,
 809,
 810,
 811,
 812,
 813,
 1002,
 1003,
 1004,
 1005,
 1006,
 1007,
 1008,
 1009,
 1010,
 1011,
 1012,
 1013,
 1202,
 1203,
 1204,
 1205,
 1206,
 1207,
 1208,
 1209,
 1210,
 1211,
 1212,
 1213,
 1402,
 1403,
 1404,
 1405,
 1406,
 1407,
 1408,
 1409,
 1410,
 1411,
 1412,
 1413,
 1602,
 1603,
 1604,
 1605,
 1606,
 1607,
 1608,
 1609,
 1610,
 1611,
 1612,
 1613,
 1802,
 1803,
 1804,
 1805,
 1806,
 1807,
 1808,
 1809,
 1810,
 1811,
 1812,
 1813,
 2002,
 2003,
 2004,
 2005,
 2006,
 2007,
 2008,
 2009,
 2010,
 2011,
 2012,
 2013,
 2202,
 2203,
 2204,
 2205,
 2206,
 2207,
 2208,
 2209,
 2210,
 2211,
 2212,
 2213,
 2402,
 2403,
 2404,
 2405,
 2406,
 2407,
 2408,
 2409,
 2410,
 2411,
 241

In [22]:
final_df = sorted_scores_index.loc[filter_indices]

In [23]:
final_df

,customer_id,article_id,predicted_score
0,0,2067,2.151094
1,0,1871,1.954697
2,0,2257,1.595641
3,0,6262,1.275684
4,0,1880,1.232110
...,...,...,...
254063559,1371979,55,-0.270533
254063560,1371979,353,-0.280082
254063561,1371979,373,-0.281689
254063562,1371979,266,-0.301477


In [24]:
final_final_df = final_df.drop("predicted_score", axis=1)

In [25]:
dataset.create_submission(final_final_df, sub_name="lmluw_dataset")